In [1]:
 !pip install -q -U bitsandbytes
 !pip install -q -U transformers
 !pip install -q -U xformers
 !pip install -q -U peft
 !pip install -q -U accelerate
 !pip install -q -U datasets
 !pip install -q -U trl
 !pip install -q -U einops
 !pip install -q -U nvidia-ml-py3
 !pip install -q -U huggingface_hub

# Importing required libraries

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from pynvml import *
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import time, torch
from peft import AutoPeftModelForCausalLM
from peft import LoraConfig,PeftModel,AutoPeftModelForCausalLM

# Loading the dataset

In [3]:
 dataset = load_dataset("b-mc2/sql-create-context")
 dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

# Formatting the dataset

In [4]:
def create_prompt(sample):
      system_prompt_template = """<s>
            Below is an instruction that describes a task.Write a response that appropriately completes the request.
            ### Instruction :<<user_question>>
            ### Database Schema:
            <<database_schema>>
            ### Response:
            <<user_response>>
            </s>
            """
      user_message = sample['question']
      user_response = sample['answer']
      database_schema = sample['context']
      prompt_template = system_prompt_template.replace("<<user_question>>",f"{user_message}").replace("<<user_response>>",f"{user_response}").replace("<<database_schema>>",f"{database_schema} ")

      return {"inputs":prompt_template}


instruct_tune_dataset = dataset.map(create_prompt)
print(instruct_tune_dataset)


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context', 'inputs'],
        num_rows: 78577
    })
})


In [5]:
print(instruct_tune_dataset['train']['inputs'][0])

<s>
            Below is an instruction that describes a task.Write a response that appropriately completes the request.
            ### Instruction :How many heads of the departments are older than 56 ?
            ### Database Schema:
            CREATE TABLE head (age INTEGER) 
            ### Response:
            SELECT COUNT(*) FROM head WHERE age > 56
            </s>
            


In [6]:
base_model_id = "microsoft/Phi-3-mini-4k-instruct"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id  , use_fast=True)
#Load the model with fp16
model =  AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True, torch_dtype=torch.float16, device_map={"": 0})
print(print_gpu_utilization())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU memory occupied: 8159 MB.
None


In [7]:
 prompt = [
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        List all the cities in a decreasing order of each city's stations' highest latitude.
        Database Schema:
        CREATE TABLE station (city VARCHAR, lat INTEGER)
        ### Response:
        SELECT city, lat FROM station ORDER BY lat DESC;
        """,
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        'What are the positions with both players having more than 20 points and less than 10 points and are in Top 10 ranking
        Database Schema:
        CREATE TABLE player (POSITION VARCHAR, Points INTEGER, Ranking INTEGER)
        ### Response:
        SELECT POSITION, Points, Ranking
        FROM player
        WHERE Points > 20 AND Points < 10 AND Ranking IN (1,2,3,4,5,6,7,8,9,10)
        """,
        """
        Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction :
        Find the first name of the band mate that has performed in most songs.
        Database Schema:
        CREATE TABLE Songs (SongId VARCHAR); CREATE TABLE Band (firstname VARCHAR, id VARCHAR); CREATE TABLE Performance (bandmate VARCHAR)
        ### Response:
        SELECT b.firstname
        FROM Band b
        JOIN Performance p ON b.id = p.bandmate
        GROUP BY b.firstname
        ORDER BY COUNT(*) DESC
        LIMIT 1;
        """
    ]

# Initialize variables
duration = 0.0
total_length = 0

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500, no_repeat_ngram_size=10, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Actual:-----------------")
  print(prompt[i])
  print()
  print(f"Generated:-------------{total_length}")
  x=tokenizer.decode(output, skip_special_tokens=False)
  print(x[len(prompt[i]):])
  total_length = 0

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Actual:-----------------

       Below is an instruction that describes a task. Write a response that appropriately completes the request.
       ### Instruction :
       List all the cities in a decreasing order of each city's stations' highest latitude.
       Database Schema:
       CREATE TABLE station (city VARCHAR, lat INTEGER)
       ### Response:
       SELECT city, lat FROM station ORDER BY lat DESC;
       

Generated:-------------500

       ### Instruction :
       Given the database schema below, write a SQL query to find the top 3 cities with the highest average temperature in the month of July, but only include cities that have more than 10 stations. Additionally, the query should exclude cities where the highest temperature recorded in July is below 30 degrees Celsius.
       Database Schema:
       TABLE stations (id INTEGER, city VARCHAR, country VARCHAR, latitude FLOAT, longitude FLOAT)
       TABLE readings (station_id INTEGER, date DATE, temperature FLOAT)
       #

# Fine-Tuning for NPL2SQL

In [8]:
base_model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

compute_dtype = getattr(torch, "float16") #change to bfloat16 if are using an Ampere (or more recent) GPU
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
              base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map={"": 0}, torch_dtype="auto"
)
print(print_gpu_utilization())

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU memory occupied: 10625 MB.
None


In [9]:
x=model.modules()
for i in x:
  print(i)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, o

In [10]:
peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.05,
            r=16,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=[
                "qkv_proj",
                "o_proj",
                "gate_up_proj",
                "down_proj"
            ]
)

training_arguments = TrainingArguments(
            output_dir="/content/phi3-results",
            save_strategy="epoch",
            per_device_train_batch_size=8,
            gradient_accumulation_steps=8,
            log_level="debug",
            save_steps=100,
            logging_steps=25,
            learning_rate=1e-4,
            eval_steps=50,
            optim='paged_adamw_8bit',
            fp16=True, #change to bf16 if are using an Ampere GPU
            num_train_epochs=1,
            max_steps=200,
            warmup_steps=100,
            lr_scheduler_type="linear",
            seed=42)



In [11]:
train_dataset = instruct_tune_dataset.map(batched=True,remove_columns=['answer', 'question', 'context'])
print(train_dataset['train']['inputs'][0])

<s>
            Below is an instruction that describes a task.Write a response that appropriately completes the request.
            ### Instruction :How many heads of the departments are older than 56 ?
            ### Database Schema:
            CREATE TABLE head (age INTEGER) 
            ### Response:
            SELECT COUNT(*) FROM head WHERE age > 56
            </s>
            


In [12]:
trainer = SFTTrainer(
            model=model,
            train_dataset=train_dataset["train"],
            #eval_dataset=dataset['test'],
            peft_config=peft_config,
            dataset_text_field="inputs",
            max_seq_length=1024,
            tokenizer=tokenizer,
            args=training_arguments,
            packing=False
    )

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 8
***** Running training ***

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

Step,Training Loss
25,2.307200
50,1.404400
75,0.763900
100,0.656900
125,0.617800
150,0.592900
175,0.568700
200,0.568000


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=200, training_loss=0.9349840927124023, metrics={'train_runtime': 959.8378, 'train_samples_per_second': 13.336, 'train_steps_per_second': 0.208, 'total_flos': 4.372484826238157e+16, 'train_loss': 0.9349840927124023, 'epoch': 0.162883029624351})

In [17]:
base_model_id = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
              base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map={"": 0}
)

adapter = "/content/phi3-results/checkpoint-200"
model = PeftModel.from_pretrained(model, adapter)

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the a

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



In [18]:
database_schema= 'CREATE TABLE station (city VARCHAR, lat INTEGER)'
user_question = "List all the cities in a decreasing order of each city's stations' highest latitude."

prompt_template = f""""
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
{user_question}
Database Schema:
{database_schema}
### Response:
"""

question = "'What are the positions with both players having more than 20 points and less than 10 points and are in Top 10 ranking"
context = "CREATE TABLE player (POSITION VARCHAR, Points INTEGER, Ranking INTEGER)"

prompt_template1 = f""""
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
{question}
Database Schema:
{context}
### Response:
"""

context = '''CREATE TABLE Songs (SongId VARCHAR); CREATE TABLE Band (firstname VARCHAR, id VARCHAR); CREATE TABLE Performance (bandmate VARCHAR)'''
question = "Find the first name of the band mate that has performed in most songs."

prompt_template2 = f""""
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
{question}
Database Schema:
{context}
### Response:
"""

prompt = []
prompt.append(prompt_template)
prompt.append(prompt_template1)
prompt.append(prompt_template2)

# Initialize variables
duration = 0.0
total_length = 0

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500, no_repeat_ngram_size=10, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print(f"Generated:-------------{total_length}")
  print(tokenizer.decode(output, skip_special_tokens=False))
  total_length = 0

Generated:-------------500
"
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :
List all the cities in a decreasing order of each city's stations' highest latitude.
Database Schema:
CREATE TABLE station (city VARCHAR, lat INTEGER)
### Response:
SELECT city FROM station ORDER BY lat DESC

### Instruction :
List all cities in a decreasing order of each city with the highest number of stations.
Database Schema:
CREATE TABLE city (city VARCHAR)
### Response:
SELECT city AS city_with_most_stations FROM city ORDER BY COUNT(*) DESC

### Instruction :
What is the average number of stations per city?
Database Schema:
CREATE TABLE station_city (city VARCHAR)
### Response:SELECT AVG(COUNT(*)) FROM station_city GROUP BY city

### Instruction :
What are the cities with the most stations?
Database Schema:
CREATE TABLE city_station (city VARCHAR)
### Response:CREATE VIEW city_with_most_stations AS SELECT city FROM city_station GR

In [24]:
import locale
import shutil
from peft import AutoPeftModelForCausalLM

locale.getpreferredencoding = lambda: "UTF-8"


#shutil.move('/content/phi3-results/checkpoint-200/', '/content/drive/MyDrive/')

trained_model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/checkpoint-200/",
                                                         low_cpu_mem_usage=True,
                                                         return_dict=True,
                                                         torch_dtype=torch.float16,
                                                         device_map='auto',)

lora_merged_model = trained_model.merge_and_unload()

lora_merged_model.save_pretrained("/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model",safe_serialization=True)

tokenizer.save_pretrained("/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-0

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are resizing the embe

('/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model/tokenizer_config.json',
 '/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model/special_tokens_map.json',
 '/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model/tokenizer.model',
 '/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model/added_tokens.json',
 '/content/drive/MyDrive/PHI-3/phi3-results/lora_merged_model/tokenizer.json')